# Database connectivity 

in Python refers to the ability of Python applications to communicate and interact with databases. This is usually achieved using modules or libraries that provide a consistent interface between the Python application and the database system.

In [2]:
import sqlite3

# Step 1: Connect to a database named 'sample.db'. If it doesn't exist, it will be created.
conn = sqlite3.connect('C:/work_prog/GEA_IKV_Prog_2/notebooks/sample.db')

# Step 2: Create a table called `users` with an `id` and a `name`.
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
)
''')

# Step 3: Populate the table with some data.
users_data = [
    ('Alenka',),
    ('Boris',),
    ('Cene',),
]
cursor.executemany('INSERT INTO users (name) VALUES (?)', users_data)
conn.commit()

# Step 4: Query the table and retrieve all rows.
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()

# Step 5: Close the connection.
conn.close()

rows


[(1, 'Alenka'), (2, 'Boris'), (3, 'Cene')]

Demo of CRUD (Create, Read, Update, Delete) operations

In [3]:
import sqlite3

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

def create_user(conn, name):
    cursor = conn.cursor()
    cursor.execute("INSERT INTO users (name) VALUES (?)", (name,))
    conn.commit()
    return cursor.lastrowid

def read_users(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    return cursor.fetchall()

def update_user(conn, user_id, new_name):
    cursor = conn.cursor()
    cursor.execute("UPDATE users SET name = ? WHERE id = ?", (new_name, user_id))
    conn.commit()

def delete_user(conn, user_id):
    cursor = conn.cursor()
    cursor.execute("DELETE FROM users WHERE id = ?", (user_id,))
    conn.commit()


In [4]:
db_path = 'C:/work_prog/GEA_IKV_Prog_2/notebooks/sample.db'
    
# Connect to the database
conn = connect_to_db(db_path)
    
# Create a new user
new_user_id = create_user(conn, "Domen")
print(f"User 'Domen' added with ID {new_user_id}")
    
# Read all users
print("Users in the database:")
for user in read_users(conn):
    print(user)
    
# Update a user's name
update_user(conn, new_user_id, "Damjan")
print(f"\nUpdated user {new_user_id}'s name to 'Damjan'")
    
# Read all users after update
print("Users in the database after update:")
for user in read_users(conn):
    print(user)
    
# Delete a user
delete_user(conn, new_user_id)
print(f"\nDeleted user with ID {new_user_id}")
    
# Read all users after deletion
print("Users in the database after deletion:")
for user in read_users(conn):
    print(user)
    
# Close the connection
conn.close()


User 'Domen' added with ID 4
Users in the database:
(1, 'Alenka')
(2, 'Boris')
(3, 'Cene')
(4, 'Domen')

Updated user 4's name to 'Damjan'
Users in the database after update:
(1, 'Alenka')
(2, 'Boris')
(3, 'Cene')
(4, 'Damjan')

Deleted user with ID 4
Users in the database after deletion:
(1, 'Alenka')
(2, 'Boris')
(3, 'Cene')


# Connect to postgres database

In [5]:
import psycopg2

def connect_to_postgres(dbname, user, password, host="localhost", port="5432"):
    try:
        # Connect to the PostgreSQL server
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        
        # Create a new cursor
        cursor = conn.cursor()
        
        # Print PostgreSQL version
        cursor.execute("SELECT version();")
        db_version = cursor.fetchone()
        print(f"Connected to: {db_version[0]}")
        
        # Close the cursor and connection
        cursor.close()
        conn.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
        return None

# Call the function
connect_to_postgres("gea_ikv_test", "bojan", "bojan123", "localhost", "5432")


Connected to: PostgreSQL 15.1, compiled by Visual C++ build 1914, 64-bit


In [6]:
import pandas as pd

def connect_and_fetch_data():
    # PostgreSQL server connection parameters
    host = "localhost"  
    dbname = "ashrae"
    user = "bojan"
    password = "bojan123"
    port = "5432"
    
    # Connection string
    conn_string = f"host={host} dbname={dbname} user={user} password={password} port={port}"

    try:
        # Connect to the PostgreSQL server
        conn = psycopg2.connect(conn_string)

        # Create a new cursor
        cur = conn.cursor()

        # Execute a query
        cur.execute("SELECT * FROM breast_cancer")

        # Fetch all rows from the table
        rows = cur.fetchall()

        # Convert to a pandas DataFrame for nicer display
        df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])

        # Display the DataFrame
        print(df)

        # Close the cursor and connection
        cur.close()
        conn.close()
        return df

    except Exception as e:
        print(f"An error occurred: {e}")

df = connect_and_fetch_data()


      id            class_name    age menopause tumor_size inv_nodes  \
0      0  no-recurrence-events  30-39   premeno      30-34       0-2   
1      1  no-recurrence-events  40-49   premeno      20-24       0-2   
2      2  no-recurrence-events  40-49   premeno      20-24       0-2   
3      3  no-recurrence-events  60-69      ge40      15-19       0-2   
4      4  no-recurrence-events  40-49   premeno        0-4       0-2   
..   ...                   ...    ...       ...        ...       ...   
281  281     recurrence-events  30-39   premeno      30-34       0-2   
282  282     recurrence-events  30-39   premeno      20-24       0-2   
283  283     recurrence-events  60-69      ge40      20-24       0-2   
284  284     recurrence-events  40-49      ge40      30-34       3-5   
285  285     recurrence-events  50-59      ge40      30-34       3-5   

    node_caps  deg_malig breast breast_quad irradiat  
0          no          3   left    left_low       no  
1          no          2 

In [7]:
from tabulate import tabulate

# Use tabulate to display the DataFrame in a table format
print(tabulate(df, headers='keys', tablefmt='grid'))


+-----+------+----------------------+-------+-------------+--------------+-------------+-------------+-------------+----------+---------------+------------+
|     |   id | class_name           | age   | menopause   | tumor_size   | inv_nodes   | node_caps   |   deg_malig | breast   | breast_quad   | irradiat   |
+=====+======+======================+=======+=============+==============+=============+=============+=============+==========+===============+============+
|   0 |    0 | no-recurrence-events | 30-39 | premeno     | 30-34        | 0-2         | no          |           3 | left     | left_low      | no         |
+-----+------+----------------------+-------+-------------+--------------+-------------+-------------+-------------+----------+---------------+------------+
|   1 |    1 | no-recurrence-events | 40-49 | premeno     | 20-24        | 0-2         | no          |           2 | right    | right_up      | no         |
+-----+------+----------------------+-------+-------------

In [8]:
# Example of using pandas styling
styled_df = df.style.background_gradient(cmap='viridis', subset=["deg_malig"]) \
    .highlight_max(color='lightgreen', subset=["tumor_size"]) \
    .highlight_min(color='yellow', subset=["class_name"]) \
    .format("{:.2f}", subset=[]) \
    .set_properties(**{'text-align': 'center'})

# Display the styled DataFrame
styled_df
